In [137]:
import pandas as pd
import json
from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from langchain.chains.llm import LLMChain

In [138]:
pd.set_option('display.max_colwidth', None)  # Set the display width to maximum.

# 1. Ticke Categorization

In [139]:
df = pd.read_csv("Support_ticket_text_data_mid_term.csv")
df.head()

,support_tick_id,support_ticket_text
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly."
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem."
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?"


In [140]:
llm = OllamaLLM(model="gemma:2b", temperature=0) # because we need consistent output

In [141]:
template = """
Classify the following support ticket into these categories:
- Technical issues
- Hardware issues
- Data recovery

Only return a json in this format: {{"category": "your_category_preiction"}}
"""

In [142]:
prompt = PromptTemplate(input_variables=["ticket"], template=template)

In [143]:
chain = LLMChain(llm=llm, prompt=prompt) # LLMChain has been deprecated

In [144]:
def classify_ticket(text):
    result = chain.run(ticket=text)
    return result

df["classification_output"] = df["support_ticket_text"].apply(classify_ticket)

In [145]:
df

,support_tick_id,support_ticket_text,classification_output
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.","{""category"": ""Technical issues""}"
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue","{""category"": ""Technical issues""}"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?","{""category"": ""Technical issues""}"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.","{""category"": ""Technical issues""}"
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?","{""category"": ""Technical issues""}"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,"{""category"": ""Technical issues""}"
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.","{""category"": ""Technical issues""}"
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing frequent crashes. Help me diagnose and resolve this hardware issue.","{""category"": ""Technical issues""}"
8,ST2023-014,My external hard drive isn't being recognized by my computer. I need assistance recovering vital data stored on it.,"{""category"": ""Technical issues""}"
9,ST2023-015,"The graphics card in my gaming laptop seems to be malfunctioning, causing poor gaming performance. Can you assist in diagnosing and fixing this hardware issue?","{""category"": ""Technical issues""}"


- gemma:2b classified everything into technical issues. 
- This usually happens due to:
    - Weak prompt instructions
    - Model too small to understand context well (2B is very small)
    - Lack of few-shot examples (we didn’t show examples of each category)

## Ticket Categorization with improved prompt

In [146]:
template = """
You are a support assistant. Categorize the support ticket below into one of the following:

- Technical issues
- Hardware issues
- Data recovery

Respond in JSON format only: {{ "category": "your_category_prediction" }}

Examples:
Ticket: "My laptop crashes when I open Zoom."
Output: {{ "category": "Technical issues" }}

Ticket: "The hard drive is making clicking noises and won't boot."
Output: {{ "category": "Hardware issues" }}

Ticket: "I accidentally deleted all my files from the USB. Can I get them back?"
Output: {{ "category": "Data recovery" }}

Now classify this:
Ticket: "{ticket}"
Output:
"""

In [147]:
prompt = PromptTemplate.from_template(template)
llm = OllamaLLM(model="gemma:2b", temperature=0)

chain = prompt | llm

def classify_ticket(text):
    result = chain.invoke({"ticket": text})
    return result

df["classification_output"] =df["support_ticket_text"].apply(classify_ticket)

In [148]:
df

,support_tick_id,support_ticket_text,classification_output
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.","{ ""category"": ""Technical issues"" }"
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue","{ ""category"": ""Hardware issues"" }"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?","{ ""category"": ""Data recovery"" }"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.","{ ""category"": ""Technical issues"" }"
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?","{ ""category"": ""Technical issues"" }"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,"{ ""category"": ""Technical issues"" }"
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.","{ ""category"": ""Technical issues"" }"
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing frequent crashes. Help me diagnose and resolve this hardware issue.","{ ""category"": ""Hardware issues"" }"
8,ST2023-014,My external hard drive isn't being recognized by my computer. I need assistance recovering vital data stored on it.,"{ ""category"": ""Data recovery"" }"
9,ST2023-015,"The graphics card in my gaming laptop seems to be malfunctioning, causing poor gaming performance. Can you assist in diagnosing and fixing this hardware issue?","{ ""category"": ""Technical issues"" }"


## Importance of prompt engineering - Explaination from ChatGPT

Now classify this:
Ticket: "{ticket}"
Output:

this line in our prompt. i ran without it, the results are inaccurate, and i ran multiple times each time the output changes. But when i ran with this line, even though i ran twice, it properly classified as data recovery, hardware issues etc. what's going on.
is our chain replacing that {ticket} with our ticket? 



Yes — **exactly**. When you run:

```python
result = chain.invoke({"ticket": "My screen is cracked and unresponsive"})
```

LangChain replaces the `{ticket}` in your prompt template with:

```text
My screen is cracked and unresponsive
```

---

### 🔧 How it works under the hood:

Your prompt template:

```python
PromptTemplate.from_template("""
... examples ...

Now classify this:
Ticket: "{ticket}"
Output:
""")
```

LangChain internally does this:

```python
filled_prompt = prompt.format(ticket="My screen is cracked and unresponsive")
```

So the final prompt sent to the LLM looks like:

```text
You are a support assistant. Categorize the support ticket...

Examples:
Ticket: "My laptop crashes when I open Zoom."
Output: {"category": "Technical issues"}

...

Now classify this:
Ticket: "My screen is cracked and unresponsive"
Output:
```

That’s what gets passed to the model.

---



Exactly — that’s the magic *and* the challenge of LLMs.

LLMs aren't truly “intelligent” in the human sense; they're **pattern-completers**. So tiny differences in prompt phrasing, punctuation, or even spacing can cause:

* A totally different interpretation,
* Random or unstructured outputs,
* Or dead-accurate predictions — *depending on how well your pattern matches its training data*.

That’s why prompt engineering is a real, in-demand skill now.

---


# 2. Ticket Categorization and Tag Generation

- We are combining step 1 and adding a new step of tag generation too.

- In our case, only prompt is going to improve as we want to genearte new features.
- Underlying basics remains the same.
- So let's build it as a fucntion, so we don't have to redefine it each and every time.

In [149]:
def llm_chain(prompt_template: str, model="gemma:2b", ):
    prompt = PromptTemplate.from_template(prompt_template)
    llm = OllamaLLM(model=model, temperature=0)
    chain = prompt | llm
    return chain

## 🎯 LangChain treats anything inside `{}` as a **required variable** — so if your prompt has:

```python
Ticket: "{ticket}"
Category: "{category}"
```

…it expects both `ticket` and `category` as inputs unless you **escape** them like `{{category}}`.

---

### 🧠 Why LangChain is amazing:

* **Composable chains** → like building blocks
* Handles prompt templates, LLMs, tools, memory, and retrieval
* Makes switching between models/local/remote seamless
* Ideal for **production-grade** GenAI apps

You’re already using it the right way.



In [150]:
template = """
You are a support assistant.

First, classify the support ticket into one of these categories:
- Technical issues
- Hardware issues
- Data recovery

Then, extract 1 to 3 relevant tags from the support ticket that describes its main topics.

Return you output as a JSON object with the following keys:
{{
    "category": "your_category_prediction",
    "tags": ["tag1", "tag2", "tag3"],
}}

Examples:

Ticket: "My laptop crashes when i open Zoom."
Output: {{
    "category": "Technical issues",
    "tags": ["Zoom", "crash", "video call"]
}}

Ticket: "My external hard disk is not getting detected."
Output: {{
    "category": "Hardware issues",
    "tags": ["external hard drive", "detection", "hardware"]
}}

Ticket: "I deleted important project files and need to recover them."
Output: {{
    "category": "Data recovery",
    "tags": ["file deletion", "project", "recovery"]
}}

Now process this ticket:
Ticket: "{ticket}"
Output:
"""

In [151]:
chain = llm_chain(prompt_template=template)

def classify_and_tag(text):
    result = chain.invoke({"ticket": text})
    return result

df["classification_and_tag_output"] = df["support_ticket_text"].apply(classify_and_tag)

In [152]:
df

,support_tick_id,support_ticket_text,classification_output,classification_and_tag_output
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""internet slow down"", ""disconnections"", ""work efficiency""]\n}"
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue","{ ""category"": ""Hardware issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""laptop"", ""refusal to start"", ""presentation""]\n}"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?","{ ""category"": ""Data recovery"" }","{\n ""category"": ""Data recovery"",\n ""tags"": [""file deletion"", ""project"", ""recovery""]\n}"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""Wi-Fi"", ""signal"", ""weak"", ""troubleshooting""]\n}"
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""battery"", ""draining"", ""minimal use""]\n}"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,"{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""login"", ""password"", ""online banking""]\n}"
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""performance"", ""slowness"", ""productivity""]\n}"
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing frequent crashes. Help me diagnose and resolve this hardware issue.","{ ""category"": ""Hardware issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""blue screen"", ""error"", ""hardware""]\n}"
8,ST2023-014,My external hard drive isn't being recognized by my computer. I need assistance recovering vital data stored on it.,"{ ""category"": ""Data recovery"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""external hard drive"", ""not recognized"", ""data recovery""]\n}"
9,ST2023-015,"The graphics card in my gaming laptop seems to be malfunctioning, causing poor gaming performance. Can you assist in diagnosing and fixing this hardware issue?","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""graphics card"", ""gaming laptop"", ""performance""]\n}"


- The output of the model are good.
- But let's clean it and create 2 diff columns: category and tags.
- Then place the individual response under these columns.
- Because our model is returning the output in json format, we can use json library to achieve our desired format.

In [153]:
df["classification_and_tag_output"][0]

'{\n    "category": "Technical issues",\n    "tags": ["internet slow down", "disconnections", "work efficiency"]\n}'

In [154]:
json.loads(df["classification_and_tag_output"][0])

{'category': 'Technical issues',
 'tags': ['internet slow down', 'disconnections', 'work efficiency']}

In [155]:
json.loads(df["classification_and_tag_output"][0]).keys()

dict_keys(['category', 'tags'])

In [156]:
json.loads(df["classification_and_tag_output"][0])['category']

'Technical issues'

In [157]:
def clean_json_result(result, key:str):
    json_result = json.loads(result)
    key_output = json_result[key]
    return key_output

df["category"] = df["classification_and_tag_output"].apply(clean_json_result, key="category")
df["tags"] = df["classification_and_tag_output"].apply(clean_json_result, key="tags")

df

,support_tick_id,support_ticket_text,classification_output,classification_and_tag_output,category,tags
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""internet slow down"", ""disconnections"", ""work efficiency""]\n}",Technical issues,"[internet slow down, disconnections, work efficiency]"
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue","{ ""category"": ""Hardware issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""laptop"", ""refusal to start"", ""presentation""]\n}",Technical issues,"[laptop, refusal to start, presentation]"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?","{ ""category"": ""Data recovery"" }","{\n ""category"": ""Data recovery"",\n ""tags"": [""file deletion"", ""project"", ""recovery""]\n}",Data recovery,"[file deletion, project, recovery]"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""Wi-Fi"", ""signal"", ""weak"", ""troubleshooting""]\n}",Technical issues,"[Wi-Fi, signal, weak, troubleshooting]"
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""battery"", ""draining"", ""minimal use""]\n}",Technical issues,"[battery, draining, minimal use]"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,"{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""login"", ""password"", ""online banking""]\n}",Technical issues,"[login, password, online banking]"
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""performance"", ""slowness"", ""productivity""]\n}",Technical issues,"[performance, slowness, productivity]"
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing frequent crashes. Help me diagnose and resolve this hardware issue.","{ ""category"": ""Hardware issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""blue screen"", ""error"", ""hardware""]\n}",Technical issues,"[blue screen, error, hardware]"
8,ST2023-014,My external hard drive isn't being recognized by my computer. I need assistance recovering vital data stored on it.,"{ ""category"": ""Data recovery"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""external hard drive"", ""not recognized"", ""data recovery""]\n}",Technical issues,"[external hard drive, not recognized, data recovery]"
9,ST2023-015,"The graphics card in my gaming laptop seems to be malfunctioning, causing poor gaming performance. Can you assist in diagnosing and fixing this hardware issue?","{ ""category"": ""Technical issues"" }","{\n ""category"": ""Technical issues"",\n ""tags"": [""graphics card"", ""gaming laptop"", ""performance""]\n}",Technical issues,"[graphics card, gami

- let's drop classification_output and classification_and_tag_output.

In [158]:
df.drop(["classification_output", "classification_and_tag_output"], axis=1, inplace=True)
df

,support_tick_id,support_ticket_text,category,tags
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.",Technical issues,"[internet slow down, disconnections, work efficiency]"
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue",Technical issues,"[laptop, refusal to start, presentation]"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?",Data recovery,"[file deletion, project, recovery]"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.",Technical issues,"[Wi-Fi, signal, weak, troubleshooting]"
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?",Technical issues,"[battery, draining, minimal use]"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,Technical issues,"[login, password, online banking]"
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.",Technical issues,"[performance, slowness, productivity]"
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing frequent crashes. Help me diagnose and resolve this hardware issue.",Technical issues,"[blue screen, error, hardware]"
8,ST2023-014,My external hard drive isn't being recognized by my computer. I need assistance recovering vital data stored on it.,Technical issues,"[external hard drive, not recognized, data recovery]"
9,ST2023-015,"The graphics card in my gaming laptop seems to be malfunctioning, causing poor gaming performance. Can you assist in diagnosing and fixing this hardware issue?",Technical issues,"[graphics card, gaming laptop, performance]"


# 3. Ticket Categorization, Tag Generation and Priority Assignment

In [167]:
template = """
You are a support assistant.

First, classify the following support ticket into one of the following categories appropriately:
- Technical issues
- Hardware issues
- Data recovery

Ticket types that fall into Technical issues: 
* Software issues
* Compatibility issues
* Network connectivity issues
* Configuration errors

Ticket types that fall into Hardware issues:
* Physical damage
* Faulty hardware components
* Operating system issues
* Drivers issues

Ticket types that fall into Data recovery:
* Loss of unsaved data
* Corrupted files
* Deleted or lost partitions

Second, extract relevant 1 to 3 tags from the support ticket that describes it's main topics.

Third, assign a priority level based on urgency and severity of the support ticket. Use the below:
- High
- Medium
- Low

Respond in the following JSON format:
{{
  "category": "your_category_prediction",
  "tags": ["your_tag_prediction"],
  "priority": "your_priority_prediction"
}}

Example 1:
Ticket: "My monitor won't turn on and I have a deadline today."
Output:
{{
  "category": "Hardware issues",
  "tags": ["monitor", "won't turn on", "deadline"],
  "priority": "High"
}}

Example 2:
Ticket: "Need help restoring some old backup files from last year."
Output:
{{
  "category": "Data recovery",
  "tags": ["backup", "file restore", "old data"],
  "priority": "Low"
}}

Example 3:
Ticket: "Need to service my laptop within next week."
Output:
{{
  "category": "Hardware",
  "tags": ["service", "laptop", "next week"],
  "priority": "Medium"
}}

Now classify:
Ticket: "{ticket}"
Output:
"""


In [ ]:
# Generated by llama3.2
template = """
You are a support assistant.

Classify the following support ticket into one of the three categories:
- Technical issues
- Hardware issues
- Data recovery

Ticket types that fall into Technical issues:
- Software issues 
- Compatibility issues 
- Network connectivity issues 
- Configuration errors 

Ticket types that fall into Hardware issues:
- Physical damage 
- Faulty hardware components 
- Operating system issues 
- Drivers issues 

Ticket types that fall into Data recovery:
- Loss of unsaved data 
- Corrupted files 
- Deleted or lost partitions 

Extract relevant 1 to 3 tags from the support ticket that describe its 
main topics. Use natural language to describe.

Assign a priority level based on urgency and severity of the support 
ticket using the following keywords:
- High:
  urgent, critical, deadline-dependent, security threat, high-priority 
request
- Medium:
  pending, planned, moderate-urgency, routine maintenance, medium-priority 
request
- Low:
  informational, low-priority request, non-urgent, cosmetic issue, 
low-severity

Respond in the following JSON format:
{{
  "category": "your_category_prediction",
  "tags": ["your_tag_prediction"],
  "priority": "your_priority_prediction"
}}


Now classify:
Ticket: "{ticket}"
Output:
"""


In [180]:
chain = llm_chain(prompt_template=template)
df["category_tag_priority_output"] = df["support_ticket_text"].apply(classify_and_tag)

KeyboardInterrupt: 

In [ ]:
df

,support_tick_id,support_ticket_text,category,tags,category_tag_priority_output
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.",Technical issues,"[internet slow down, disconnections, work efficiency]","{\n ""category"": ""Technical issues"",\n ""tags"": [""internet"", ""slow down"", ""disruptions""],\n ""priority"": ""High""\n}"
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue",Technical issues,"[laptop, refusal to start, presentation]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""laptop"", ""refuses to start"", ""hardware issue""],\n ""priority"": ""High""\n}"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?",Data recovery,"[file deletion, project, recovery]","{\n ""category"": ""Data recovery"",\n ""tags"": [""backup"", ""file restore"", ""old data""],\n ""priority"": ""Low""\n}"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.",Technical issues,"[Wi-Fi, signal, weak, troubleshooting]","{\n ""category"": ""Technical issues"",\n ""tags"": [""Wi-Fi"", ""signal"", ""weak"", ""troubleshooting""],\n ""priority"": ""High""\n}"
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?",Technical issues,"[battery, draining, minimal use]","{\n ""category"": ""Technical issues"",\n ""tags"": [""smartphone"", ""battery"", ""draining"", ""minimal use""],\n ""priority"": ""High""\n}"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,Technical issues,"[login, password, online banking]","{\n ""category"": ""Technical issues"",\n ""tags"": [""lockout"", ""online banking"", ""password reset""],\n ""priority"": ""High""\n}"
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.",Technical issues,"[performance, slowness, productivity]","{\n ""category"": ""Technical issues"",\n ""tags"": [""performance"", ""sluggish"", ""impactful""],\n ""priority"": ""High""\n}"
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing frequent crashes. Help me diagnose and resolve this hardware issue.",Technical issues,"[blue screen, error, hardware]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""blue screen"", ""error"", ""hardware issue""],\n ""priority"": ""High""\n}"
8,ST2023-014,My external hard drive isn't being recognized by my computer. I need assistance recovering vital data stored on it.,Technical issues,"[external hard drive, not recognized, data recovery]","{\n ""category"": ""Data recovery"",\n ""tags"": [""external hard drive"", ""not recognized"", ""vital data""],\n ""priority"": ""Low""\n}"
9,ST2023-015,"The graphics card in my gaming laptop seems to be malfunctioning, causing poor gaming performance. Can you assist in diagnosing and fixing this hardware issue?",Technical issues,"[graphics card, gaming laptop, performance]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""graphics card"", ""malfunctioning"", ""gaming performance""],\n ""priority"": ""High""\n}"


## llama

In [190]:
template = """
You are a support assistant.

First, classify the support ticket into one of these categories:
- Technical issues
- Hardware issues
- Data recovery

Second, extract 1 to 3 relevant tags from the support ticket that describes its main topics.

Third, assign a priority level based on urgency and severity of the support ticket. Use the below 3:
- High
- Medium
- Low

Return you output as a JSON object with the following keys alone strictly, no additional description or words or explanation needed.:
{{
    "category": "your_category_prediction",
    "tags": ["tag1", "tag2", "tag3"],
    "priority: "your_priority_prediction"
}}

Example 1:
Ticket: "My laptop is not turning on."
Output:
{{
    "category": "Hardware issues",
    "tags": ["Laptop", "Not Turning On"],
    "priority": "High"
}}

Example 2:
Ticket: "I'm getting a runtime error in my application."
Output:
{{
    "category": "Technical issues",
    "tags": ["Runtime Error", "Application Issue"],
    "priority": "Medium"
}}

Example 3:
Ticket: "I accidentally deleted a file from my Google Drive."
Output:
{{
    "category": "Data recovery",
    "tags": ["File Deleted", "Google Drive"],
    "priority": "Low"
}}

Example 4:
Ticket: "My website is slow and taking forever to load."
Output:
{{
    "category": "Technical issues",
    "tags": ["Slow Website", "Loading Issue"],
    "priority": "Medium"
}}

Example 5:
Ticket: "I'm experiencing a network connection issue on my mobile device."
Output:
{{
    "category": "Hardware issues",
    "tags": ["Network Connection", "Mobile Device"],
    "priority": "High"
}}


Now classify this:
Ticket: "{ticket}"
Output:
"""


In [191]:
chain = llm_chain(prompt_template=template, model="llama3.2")
df["category_tag_priority_output"] = df["support_ticket_text"].apply(classify_and_tag)
df

,support_tick_id,support_ticket_text,category,tags,category_tag_priority_output
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.",Technical issues,"[internet slow down, disconnections, work efficiency]","{\n ""category"": ""Technical issues"",\n ""tags"": [""Internet Connection"", ""Disconnection Issue""],\n ""priority"": ""High""\n}"
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue",Technical issues,"[laptop, refusal to start, presentation]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""Laptop"", ""Not Turning On""],\n ""priority"": ""High""\n}"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?",Data recovery,"[file deletion, project, recovery]","{\n ""category"": ""Data recovery"",\n ""tags"": [""File Deleted"", ""Data Loss"", ""Device Lockdown""],\n ""priority"": ""High""\n}"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.",Technical issues,"[Wi-Fi, signal, weak, troubleshooting]","{\n ""category"": ""Technical issues"",\n ""tags"": [""Weak Wi-Fi Signal"", ""Troubleshooting Issue""],\n ""priority"": ""Medium""\n}"
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?",Technical issues,"[battery, draining, minimal use]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""Smartphone Battery"", ""Draining Issue""],\n ""priority"": ""High""\n}"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,Technical issues,"[login, password, online banking]","{\n ""category"": ""Technical issues"",\n ""tags"": [""Locked Out"", ""Online Banking"", ""Password Reset""],\n ""priority"": ""High""\n}"
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.",Technical issues,"[performance, slowness, productivity]","{\n ""category"": ""Technical issues"",\n ""tags"": [""Computer Performance"", ""Optimization"", ""Productivity""],\n ""priority"": ""High""\n}"
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing frequent crashes. Help me diagnose and resolve this hardware issue.",Technical issues,"[blue screen, error, hardware]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""Blue Screen Error"", ""Recurring Crash""],\n ""priority"": ""High""\n}"
8,ST2023-014,My external hard drive isn't being recognized by my computer. I need assistance recovering vital data stored on it.,Technical issues,"[external hard drive, not recognized, data recovery]","{\n ""category"": ""Data recovery"",\n ""tags"": [""External Hard Drive"", ""Not Recognized"", ""Data Recovery""],\n ""priority"": ""High""\n}"
9,ST2023-015,"The graphics card in my gaming laptop seems to be malfunctioning, causing poor gaming performance. Can you assist in diagnosing and fixing this hardware issue?",Technical issues,"[graphics card, gaming laptop, performance]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""Graphics Card"", ""Gaming Laptop"", ""Malfunctioning Hardware""],\n ""priority"": ""High""\n}"


## Chatgpt

In [192]:
template = """
You are an AI assistant for a customer support team.

Your task is to analyze the following support ticket and return a JSON object with:
- A ticket category (one of: "Technical issues", "Hardware issues", "Data recovery")
- 1 to 3 relevant tags summarizing the main topics
- A priority level: "High", "Medium", or "Low"

--- Ticket Classification Rules ---

Category:
- Technical issues: Software errors, app crashes, installation, network/configuration issues.
- Hardware issues: Device not turning on, screen damage, OS update failure, component failure.
- Data recovery: Lost files, corrupted data, accidental deletion, missing backups.

Priority:
- High: The issue causes an immediate work stoppage, data loss, or missed deadline.
- Medium: The issue impacts productivity but has a workaround or is not time-critical.
- Low: The issue is minor, cosmetic, or informational with no urgency.

⚠️ Do NOT assign "High" priority unless the user clearly mentions:
- A deadline or blocked workflow
- Data loss or security concerns
- Total inoperability of critical hardware or software

--- Examples ---

Example 1:
Ticket: "My laptop won't turn on and I have a presentation in 2 hours."
Output:
{{
  "category": "Hardware issues",
  "tags": ["laptop", "won't turn on", "presentation"],
  "priority": "High"
}}

Example 2:
Ticket: "Having trouble installing the software. Need to get it working today."
Output:
{{
  "category": "Technical issues",
  "tags": ["installation", "software", "troubleshooting"],
  "priority": "Medium"
}}

Example 3:
Ticket: "Files from a previous backup seem corrupted but I have time to reupload."
Output:
{{
  "category": "Data recovery",
  "tags": ["backup", "corrupted files"],
  "priority": "Low"
}}

Example 4:
Ticket: "My email isn't syncing, but I can still use webmail for now."
Output:
{{
  "category": "Technical issues",
  "tags": ["email", "sync issue", "webmail"],
  "priority": "Low"
}}

Example 5:
Ticket: "My computer fan is making loud noises and I'm worried it may fail soon."
Output:
{{
  "category": "Hardware issues",
  "tags": ["fan noise", "possible failure", "hardware"],
  "priority": "Medium"
}}

Now classify:
Ticket: "{ticket}"
Output:
"""


In [193]:
chain = llm_chain(prompt_template=template, model="llama3.2")
df["category_tag_priority_output"] = df["support_ticket_text"].apply(classify_and_tag)
df

,support_tick_id,support_ticket_text,category,tags,category_tag_priority_output
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.",Technical issues,"[internet slow down, disconnections, work efficiency]","Based on the provided ticket, I've analyzed the situation and categorized it as follows:\n\n{\n ""category"": ""Technical issues"",\n ""tags"": [""internet connection"", ""slow speed"", ""disconnections""],\n ""priority"": ""High""\n}\n\nHere's why:\n\n* The category is ""Technical issues"" because the issue is related to a software or network configuration problem (in this case, internet connectivity).\n* The tags are relevant topics that summarize the main concerns: slow internet speed, frequent disconnections, and their impact on work efficiency.\n* The priority level is ""High"" because the user mentions that the issue is causing major disruptions and they need assistance to resolve it promptly. There's no mention of a deadline or data loss, but the impact on productivity suggests a high-priority request."
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue",Technical issues,"[laptop, refusal to start, presentation]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""laptop"", ""won't turn on"", ""presentation""],\n ""priority"": ""High""\n}"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?",Data recovery,"[file deletion, project, recovery]","Based on the provided ticket, here is the JSON object with the requested information:\n\n{\n ""category"": ""Data recovery"",\n ""tags"": [""accidental deletion"", ""essential work documents"", ""data loss""],\n ""priority"": ""High""\n}\n\nThe category is classified as ""Data recovery"" because the user has mentioned accidental deletion of essential work documents, which indicates a significant data loss issue. The priority level is set to ""High"" due to the substantial data loss and the user's request for urgent assistance."
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.",Technical issues,"[Wi-Fi, signal, weak, troubleshooting]","{\n ""category"": ""Technical issues"",\n ""tags"": [""Wi-Fi"", ""weak signal"", ""troubleshooting""],\n ""priority"": ""Medium""\n}"
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?",Technical issues,"[battery, draining, minimal use]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""battery drain"", ""smartphone"", ""rapid discharge""],\n ""priority"": ""Medium""\n}"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,Technical issues,"[login, password, online banking]","{\n ""category"": ""Technical issues"",\n ""tags"": [""online banking"", ""password reset"", ""urgent transaction""],\n ""priority"": ""High""\n}"
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.",Technical issues,"[performance, slowness, productivity]","{\n ""category"": ""Technical issues"",\n ""tags"": [""performance"", ""sluggish"", ""optimization""],\n ""priority"": ""Medium""\n}"
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing fr

# 4. 

In [214]:
template = """
You are a support ticket analyst. Analyze the ticket and respond ONLY with a JSON object, no explanation.

Instructions:

1. **category**: One of:
   - Technical issues
   - Hardware issues
   - Data recovery

2. **tags**: Extract 1 to 3 key tags.

3. **priority**: Choose based on real urgency:
   - High: Only if user mentions deadline, system crash, or urgent need.
   - Medium: If it's disruptive but not urgent.
   - Low: If it's a minor issue, information request, or not time-sensitive.

4. **eta**: Must depend on the priority:
   - High → Immediate
   - Medium → 24 hours
   - Low → 2-3 business days

5. **response**: Provide a clear, calm instruction to the user in 1 sentence.

Only return JSON strictly in this format:(no extra text)

{{
  "category": "your_category_prediction",
  "tags": ["tag1", "tag2"],
  "priority": "your_priority_prediction",
  "eta": "your_eta_prediction",
  "response": "your_brief_instruction_to_user"
}}

--- Examples ---

Example 1:
Ticket: "My laptop won't turn on and I have a presentation in 2 hours."
Output:
{{
  "category": "Hardware issues",
  "tags": ["laptop", "won't turn on", "presentation"],
  "priority": "High"
}}

Example 2:
Ticket: "Having trouble installing the software. Need to get it working today."
Output:
{{
  "category": "Technical issues",
  "tags": ["installation", "software", "troubleshooting"],
  "priority": "Medium"
}}

Example 3:
Ticket: "Files from a previous backup seem corrupted but I have time to reupload."
Output:
{{
  "category": "Data recovery",
  "tags": ["backup", "corrupted files"],
  "priority": "Low"
}}

Example 4:
Ticket: "My email isn't syncing, but I can still use webmail for now."
Output:
{{
  "category": "Technical issues",
  "tags": ["email", "sync issue", "webmail"],
  "priority": "Low"
}}

Example 5:
Ticket: "My computer fan is making loud noises and I'm worried it may fail soon."
Output:
{{
  "category": "Hardware issues",
  "tags": ["fan noise", "possible failure", "hardware"],
  "priority": "Medium"
}}

Analyze the following ticket and respond:
Ticket: "{ticket}"
Output:
"""


In [215]:
chain = llm_chain(prompt_template=template, model="llama3.2")
df["category_tag_priority_output"] = df["support_ticket_text"].apply(classify_and_tag)
df

,support_tick_id,support_ticket_text,category,tags,category_tag_priority_output
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.",Technical issues,"[internet slow down, disconnections, work efficiency]","{\n ""category"": ""Technical issues"",\n ""tags"": [""internet"", ""slow connection"", ""disruptions""],\n ""priority"": ""High"",\n ""eta"": ""Immediate"",\n ""response"": ""Our team will work to resolve your internet connectivity issue as soon as possible. Please contact us again once the issue is resolved.""\n}"
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue",Technical issues,"[laptop, refusal to start, presentation]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""laptop"", ""won't turn on"", ""presentation""],\n ""priority"": ""High"",\n ""eta"": ""Immediate"",\n ""response"": ""Please contact our technical support team immediately to assist with resolving the hardware issue and ensuring your presentation is ready for tomorrow.""\n}"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?",Data recovery,"[file deletion, project, recovery]","{\n ""category"": ""Data recovery"",\n ""tags"": [""data loss"", ""deleted files""],\n ""priority"": ""High"",\n ""eta"": ""Immediate"",\n ""response"": ""Please wait for our assistance to recover your deleted documents. We will guide you through the process and ensure minimal disruption to your work.""\n}"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.",Technical issues,"[Wi-Fi, signal, weak, troubleshooting]","{\n ""category"": ""Technical issues"",\n ""tags"": [""Wi-Fi"", ""weak signal""],\n ""priority"": ""Medium"",\n ""eta"": ""24 hours"",\n ""response"": ""Please try restarting your router and modem, then run a network speed test to see if the issue persists.""\n}"
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?",Technical issues,"[battery, draining, minimal use]","{\n ""category"": ""Hardware issues"",\n ""tags"": [""smartphone"", ""battery drain""],\n ""priority"": ""Medium"",\n ""eta"": ""24 hours"",\n ""response"": ""Please visit our website for troubleshooting guides on common battery drain causes and follow the steps to resolve the issue.""\n}"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,Technical issues,"[login, password, online banking]","{\n ""category"": ""Technical issues"",\n ""tags"": [""online banking"", ""password reset"", ""urgent transaction""],\n ""priority"": ""High"",\n ""eta"": ""Immediate"",\n ""response"": ""Please click on the 'Forgot Password' link and follow the prompts to reset your password.""\n}"
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.",Technical issues,"[performance, slowness, productivity]","{\n ""category"": ""Technical issues"",\n ""tags"": [""computer"", ""performance"", ""optimization""],\n ""priority"": ""Medium"",\n ""eta"": ""24 hours"",\n ""response"": ""Please provide more details about your computer's specifications and the tasks you're experiencing performance issues with, so we can assist you in optim

In [218]:
def clean_json_result(result, key:str):
    if "{" not in result:
        result = "{" + result
    elif "}" not in result:
        result = result + "}"
    elif "{" not in result and "}" not in result:
        result = "{" + result + "}"
    else:
        pass

    json_result = json.loads(result)
    key_output = json_result[key]
    return key_output

In [ ]:
df["category"] = df["category_tag_priority_output"].apply(lambda x: clean_json_result(x, key="category"))
df["tags"] = df["category_tag_priority_output"].apply(clean_json_result, key="tags")
df["priority"] = df["category_tag_priority_output"].apply(clean_json_result, key="priority")
df["eta"] = df["category_tag_priority_output"].apply(clean_json_result, key="eta")
df["response"] = df["category_tag_priority_output"].apply(clean_json_result, key="response")

df.drop("category_tag_priority_output", axis=1, inplace=True)


In [220]:
df

,support_tick_id,support_ticket_text,category,tags,priority,eta,response
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.",Technical issues,"[internet, slow connection, disruptions]",High,Immediate,Our team will work to resolve your internet connectivity issue as soon as possible. Please contact us again once the issue is resolved.
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue",Hardware issues,"[laptop, won't turn on, presentation]",High,Immediate,Please contact our technical support team immediately to assist with resolving the hardware issue and ensuring your presentation is ready for tomorrow.
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?",Data recovery,"[data loss, deleted files]",High,Immediate,Please wait for our assistance to recover your deleted documents. We will guide you through the process and ensure minimal disruption to your work.
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.",Technical issues,"[Wi-Fi, weak signal]",Medium,24 hours,"Please try restarting your router and modem, then run a network speed test to see if the issue persists."
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?",Hardware issues,"[smartphone, battery drain]",Medium,24 hours,Please visit our website for troubleshooting guides on common battery drain causes and follow the steps to resolve the issue.
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,Technical issues,"[online banking, password reset, urgent transaction]",High,Immediate,Please click on the 'Forgot Password' link and follow the prompts to reset your password.
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.",Technical issues,"[computer, performance, optimization]",Medium,24 hours,"Please provide more details about your computer's specifications and the tasks you're experiencing performance issues with, so we can assist you in optimizing its settings."
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing frequent crashes. Help me diagnose and resolve this hardware issue.",Hardware issues,"[blue screen, recurring crash, PC]",High,Immediate,Please provide the exact error message and any recent system updates or driver changes for immediate assistance.
8,ST2023-014,My external hard drive isn't being recognized by my computer. I need assistance recovering vital data stored on it.,Data recovery,"[external hard drive, not recognized, data recovery]",High,Immediate,Please connect the external hard drive to a different computer and try recovering your data using a different software or method.
9,ST2023-015,"The graphics card in my gaming laptop seems to be malfunctioning, causing poor gaming performance. Can you assist in diagnosing and fixing this hardware issue?",Hardware issues,"[graphics card, gaming laptop, malfunctioning]",Medium,24 hours,"Please provide more details about the error message or symptoms you're experiencing, and we'll assist in diagnosing and potentially fixing the issue."
